In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# loss 그래프를 그리기 위한 도구 설치
!pip install tensorboardX

     |████████████████████████████████| 124 kB 4.3 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import json
import copy
from itertools import chain
from keras.datasets import mnist
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorboardX import SummaryWriter
writer = SummaryWriter()

In [ ]:
# 불러와야 하는 파일
# 1. 라운지_트루라벨_3.csv
# 2. 건물 인코딩.csv
# 3. 라운지 사용자 5000 최종.csv
# 4. 라운지 데이터 최종.csv
# 5. 건물별 이동시간 최종.csv

#	사용자	위치	시간	사용자 수	책상 높이	콘센트 필요도	컴퓨터 필요도	정수기/화장실 유무
raw_data_users = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지 사용자 5000 최종.csv',encoding='euc-kr')

#	라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	혼잡도  건물
raw_data_lounge = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지 데이터 최종.csv',encoding='euc-kr')
raw_data_lounge['건물'] = raw_data_lounge['건물'].replace({' ':''}, regex=True)

#건물-건물
distance_buildings = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/건물별 이동시간 최종.csv',encoding='utf-8')

lounge_truedata = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지_트루라벨_3.csv',encoding='euc-kr')
encoding_df = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/건물 인코딩.csv')

lounge_truedata.score = lounge_truedata.score.apply(eval)
lounge_truedata.score = lounge_truedata.score.apply(np.array)
print(len(lounge_truedata.score[1]))
lounge_truedata = lounge_truedata.to_numpy()


239
     lounge_index        건물         라운지  위치(층)  책상 높이  ...  창문  컴퓨터  정수기  화장실  혼잡도
0               0   4.18기념관   4.18기념관-1     -2      1  ...   1    6    0    1  NaN
1               1   4.18기념관   4.18기념관-2     -1      1  ...   1    0    1    1  NaN
2               2   4.18기념관   4.18기념관-3      0      1  ...   0    0    0    0  NaN
3               3   4.18기념관   4.18기념관-4     -2      1  ...   0    0    0    1  NaN
4               4     CJ법학관     CJ법학관-1      0      1  ...   1    0    1    1  NaN
..            ...       ...         ...    ...    ...  ...  ..  ...  ...  ...  ...
234           234  현대자동차경영관  현대자동차경영관-6     -1      1  ...   1    4    1    0  NaN
235           235  현대자동차경영관  현대자동차경영관-7      5      1  ...   1    0    1    1  NaN
236           236     환경실험관     환경실험관-1      2      1  ...   1    0    0    1  NaN
237           237     환경실험관     환경실험관-2      3      1  ...   1    0    0    1  NaN
238           238     환경실험관     환경실험관-3      3      1  ...   0    7    0    1  NaN


In [ ]:
#encoding raw dataframes

def df_completion(user):
    user_building = user['위치']
    distance = distance_buildings[['building', user_building]].copy()
    distance.rename(columns = {user_building: '거리'}, inplace = True)

    #라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	혼잡도	건물	***거리
    lounge_with_distance = pd.merge(raw_data_lounge, distance, left_on= '건물', right_on='building', how ='left' ).drop(columns=['building'])

    
    day = str(pd.to_datetime(user['시간'].split('T')[0]).dayofweek)
    user_c = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_월.csv')

    #find day
    if day==1:
        user_c =  pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_화.csv')
    elif day==2:
        user_c = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_수.csv')
    elif day==3:
        user_c = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_목.csv')
    elif day==4:
        user_c = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_금.csv')

    range = pd.date_range("09:00", "23:30", freq="30min").strftime('%H:%M:%S')
    dt = pd.to_datetime(user['시간']).round('30min')
    user_c_r = range.get_loc(str(dt).split(' ')[1].split('+')[0])

    #라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	***혼잡도	건물    거리
    lounge_with_congestion = lounge_with_distance
    lounge_with_congestion['혼잡도']=lounge_with_congestion.apply(lambda row: user_c[row.건물][user_c_r], axis=1)

    final_lounge = pd.merge(lounge_with_congestion, encoding_df, left_on= '건물', right_on='building', how ='left')
    final_lounge = final_lounge.drop(['건물','building','라운지','위치(층)'], axis =1)
    final_lounge = final_lounge.rename({'encoding':'건물', 'lounge_index':'라운지'},axis=1)

    final_user = pd.merge(raw_data_users, encoding_df, left_on= '위치', right_on='building', how ='left')
    final_user = final_user.drop(['위치','building','시간'], axis =1)
    final_user = final_user.rename({'encoding':'건물'},axis=1)

    return final_lounge, final_user

lounge_df, user_df = df_completion(raw_data_users.iloc[100])
print(lounge_df)
print(user_df)

     라운지  책상 높이  테이블 수  의자 수  냉난방장치  콘센트 수  ...  컴퓨터  정수기  화장실       혼잡도     거리  건물
0      0      1      2     6      0      4  ...    6    0    1  0.313718   99.6  19
1      1      1      1     2      0      4  ...    0    1    1  0.313718   99.6  19
2      2      1      6    30      1      4  ...    0    0    0  0.313718   99.6  19
3      3      1      6    24      0      7  ...    0    0    1  0.313718   99.6  19
4      4      1      7    14      1      3  ...    0    1    1  0.351093  169.0  28
..   ...    ...    ...   ...    ...    ...  ...  ...  ...  ...       ...    ...  ..
234  234      1     15    60      0      5  ...    4    1    0  0.000000  243.7  32
235  235      1     15    15      1      2  ...    0    1    1  0.000000  243.7  32
236  236      1     13    65      0      2  ...    0    0    1  0.481289  123.7  69
237  237      1     15    15      0      1  ...    0    0    1  0.481289  123.7  69
238  238      1     13    52      0      1  ...    7    0    1  0.481289  12

In [ ]:
# 모델에 넣기 위해 데이터 형식 dataframe --> numpy로 변화
np.random.seed(2)
ind = np.random.rand(len(user_df)) < 0.8
train_df = user_df[ind]
train = train_df.to_numpy()
val_df = user_df[~ind]
val = val_df.to_numpy()
lounge = lounge_df.to_numpy()

len_train = len(train)
len_lounge = len(lounge)

In [ ]:
# model 

# batch_size = total classroom

class NNCollabFiltering(nn.Module):
    def __init__(self, emb_size=10, n_hidden=100):
        super(NNCollabFiltering, self).__init__()
        self.emb_size = emb_size
        self.user_emb = nn.Embedding(100, emb_size)
        self.item_emb = nn.Embedding(600, emb_size)
        self.lin1 = nn.Linear(1800, n_hidden)
        self.lin2 = nn.Linear(n_hidden, 1)
        self.drop1 = nn.Dropout(0.1)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        U = torch.reshape(U, (239, 6 * self.emb_size))
        V = self.item_emb(v)
        V = torch.reshape(V, (239, 12 * self.emb_size))
        x = F.relu(torch.cat([U, V], dim=1))
        x = self.drop1(x)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        x = torch.flatten(x)
        return x
model = NNCollabFiltering(emb_size=100)

In [ ]:
# validation model
def validation_loss(model, unsqueeze=False):
    model.eval()
    total_loss =  0.0

    for user in range(5000-len_train):
      user_scale = np.array(val[user,1:].tolist()*239).reshape((239,6))
      users = torch.LongTensor(user_scale)
      val_index = user+len_train
      lounge_df, trash_df = df_completion(raw_data_users.iloc[val_index])
      l = lounge_df.to_numpy()
      lounges = torch.LongTensor(l[:,1:])
      TL = torch.FloatTensor(lounge_truedata[val[user,0]-1,1])
      y_hat = model(users, lounges)
      loss = F.mse_loss(y_hat, TL)
      total_loss += loss.item()
    
    avg_loss = total_loss / (5000-len_train)
    print("validation loss {:.7f}".format(avg_loss))

In [ ]:
# train model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay= 1e-6)
def train_mf(model, epochs=10, unsqueeze=False):
    model.train()
    for i in range(epochs):
      running_loss = 0.0 

      for user in range(len_train):
        optimizer.zero_grad()
        user_scale = np.array(train[user,1:].tolist()*239).reshape((239,6))
        users = torch.LongTensor(user_scale)
        lounge_df, trash_df = df_completion(raw_data_users.iloc[user])
        l = lounge_df.to_numpy()
        lounges = torch.LongTensor(l[:,1:])
        TL = torch.FloatTensor(lounge_truedata[train[user,0]-1,1].reshape((239)))
        y_hat = model(users, lounges)
        loss = F.mse_loss(y_hat, TL)
        writer.add_scalar("Loss/train", loss, i*len_train+user+1)
        loss.backward(retain_graph=True)
        running_loss += loss.item()
        optimizer.step()
        if (user) % 500 == 499 :
          print('epochs:', i, ', iter: ', user+1)
          print(running_loss/500)  
          running_loss = 0.0
    writer.flush()
    validation_loss(model, unsqueeze)

In [ ]:
# train start & train 중간 loss 출력
writer.flush()
train_mf(model, epochs=7, unsqueeze=True)

epochs: 0 , iter:  500
1.748241360425949
epochs: 0 , iter:  1000
1.2212200728058815
epochs: 0 , iter:  1500
1.1896479316949844
epochs: 0 , iter:  2000
1.101890043914318
epochs: 0 , iter:  2500
1.071766622722149
epochs: 0 , iter:  3000
1.0190252646803857
epochs: 0 , iter:  3500
0.974522844016552
epochs: 0 , iter:  4000
0.9974992358088494
epochs: 1 , iter:  500
0.9678609531521797
epochs: 1 , iter:  1000
0.9308658693432807
epochs: 1 , iter:  1500
0.9372460834383964
epochs: 1 , iter:  2000
0.7941069692969323
epochs: 1 , iter:  2500
0.6606093652248383
epochs: 1 , iter:  3000
0.5662894072532654
epochs: 1 , iter:  3500
0.5136809864342212
epochs: 1 , iter:  4000
0.46219301226735116
epochs: 2 , iter:  500
0.4255387874543667
epochs: 2 , iter:  1000
0.38810072737932205
epochs: 2 , iter:  1500
0.3355223126411438
epochs: 2 , iter:  2000
0.2900530182868242
epochs: 2 , iter:  2500
0.31369752828776837
epochs: 2 , iter:  3000
0.27177005530893805
epochs: 2 , iter:  3500
0.2458251972347498
epochs: 2 , it

In [ ]:
# 모델 저장
PATH = '/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/model_3/'

torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
'model': model.state_dict(),
'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')

In [ ]:
# 모델 불러오기
model = torch.load(PATH + 'model.pt')  
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  
checkpoint = torch.load(PATH + 'all.tar')  
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
# 라운지 9개 추출

new_user = [3, 1, 4, 5, 0,  13]

def top9(model, user, unsqueeze=False):
  with torch.no_grad():
    user_scale = np.array(user*239).reshape((239,6))
    one_user = torch.LongTensor(user_scale)
    l = lounge_df.to_numpy()
    lounges = torch.LongTensor(l[:,1:])
    out = model(one_user, lounges).numpy()
    print(out)
    sort_out = np.argsort(out)
    print(sort_out)
    final = np.argsort(-sort_out)
    print(final)
    top9 = final[:9]
    print(top9)
    return top9

top9_index = top9(model, new_user, unsqueeze=True)


top9_df = pd.DataFrame(top9_index, columns =['lounge_index'])
print(top9_df)

top9 = pd.merge(top9_df, lounge_df, left_on= 'lounge_index', right_on='라운지', how ='left')
print(top9)

output = top9[['건물', '라운지']].copy().to_records(index=False)
print(output)


[6.926393  5.8087544 6.5154033 6.30055   6.410085  6.9163394 6.2371817
 5.8771296 5.787267  5.864695  6.003094  7.0662246 5.5169005 4.4126697
 4.9704175 4.2053447 4.3767376 4.3252935 4.4497705 4.1954894 4.0345206
 3.9116042 3.8154883 3.1527174 7.891381  8.153576  7.624625  6.464885
 7.8311334 7.159326  4.0275617 4.199926  4.533246  4.360554  4.1686797
 4.2628036 3.1502118 3.6647983 3.2585378 4.1191106 4.5642114 5.647385
 4.882525  5.386312  3.845696  4.1843085 3.8311515 4.8887467 4.5325694
 6.3571873 6.927006  6.754441  4.8096395 6.2450624 6.0353293 6.2622886
 5.4684052 5.818083  5.552215  6.6462255 6.0463552 5.8456483 5.5090175
 5.8612704 5.2118945 5.99673   6.745468  5.713326  5.7030454 5.738502
 4.343372  4.2333226 6.63751   6.098305  5.8622847 6.45756   5.6270094
 5.804439  6.7525916 6.870896  5.4741836 5.9141245 2.918114  4.3454037
 4.4766636 4.5055537 4.443959  5.075812  5.434859  5.4968886 4.3145223
 4.7029915 4.166806  4.851444  3.9981139 3.1879573 5.333201  6.552266
 7.07409  

'\n    print(out.size())\n    out= out.reshape(1,82)\n    print(out.size())\n    out = nn.Softmax(dim=1)(out)\n    print(out)\n'

In [ ]:
# 학습 과정 중 저장해둔 모든 loss 그래프화
writer.close()
%reload_ext tensorboard
%tensorboard --logdir runs 

Launching TensorBoard...